<a href="https://colab.research.google.com/github/yomnahisham/foodwaste/blob/main/food_waste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CSCE 2202: FoodWaste Course Project**
This is an in-progress Colab of the implementation of the course project for Analysis (Fall 2025)

Team 3: Abdelrahman Abdelbaky, Abdelrahman Osama, Mohamed Anan, and Yomna Othman

Current Milestone: MS1 (Date: 2025/11/30)

In [ ]:
print("hello world")

hello world


*Global Imports*

In [ ]:
import numpy as np
import random
import pandas as pd
from typing import List, Dict, Optional

**Restaurant APIs**

In [ ]:
# %%writefile restaurant_api.py

class Restaurant:
  def __init__(self, restaurant_id, name, category):
      self.restaurant_id = restaurant_id
      self.name = name
      self.category = category
      self.price = 0.0

      self.rating = 0.0
      self.number_of_ratings = 0

      self.est_inventory=0
      self.actual_inventory=0

      self.accuracy_score = 1.0
      self.inventory_history=[] # list of (date, est, actual)

      self.reservation_count = 0
      self.completed_order_count=0
      self.cancellation_count=0

      self.exposure_count=0

  def update_daily_supply(self, new_price, new_inventory):
    self.price = new_price
    self.est_inventory = new_inventory
    # self.reservation_count=0 #cleaned in reset_daily_counters() below


  def receive_rating(self,rating):
    n =self.number_of_ratings
    new_rating = self.rating + (rating - self.rating)/(n+1)
    self.number_of_ratings = n+ 1
    self.rating = new_rating

  def reserve_order(self):
    if self.reservation_count<self.est_inventory:
      self.reservation_count+=1

  def finalize_daily_inventory(self, date, actual_inventory):
    self.actual_inventory = actual_inventory
    self.inventory_history.append((date,self.est_inventory, actual_inventory))
    self.calculate_accuracy()

  #This is at the end of the day when we review what orders were cancelled and what was delivered
  def add_daily_summary(self, ordered, received):
    self.completed_order_count += received
    self.cancellation_count += ordered-received

  def calculate_accuracy(self):
    if not self.inventory_history:
      return 1.0   #no orders so far. accuracy = 1

    errors=[]
    for record in self.inventory_history:
      date, est, actual = record
      if est>0:
        error= max(est-actual,0)/est
        errors.append(error)
    if not errors:
      return 1.0

    errors_array = np.array(errors)
    errors_mean = np.mean(errors_array)
    #if we have less than 30 records, do not use window frame.
    if len(errors)<=30:
      accuracy = 1-errors_mean
      self.accuracy_score = accuracy
      return accuracy

    window_errors = errors[-30:]
    window_errors_mean = np.mean(window_errors)
    accuracy = 1-(0.7*window_errors_mean+0.3*errors_mean)
    self.accuracy_score = accuracy
    return accuracy

  def reset_daily_counters(self):
      """reset daily counters for a new day"""
      self.reservation_count = 0
      self.completed_order_count = 0
      self.cancellation_count = 0
      self.exposure_count = 0
      self.actual_inventory = 0


#END OF CLASS

#global store registry
_stores: Dict[int, Restaurant] = {}
_actual_inventories_for_day: Optional[Dict[int, int]] = None

def load_store_data(num_stores: int = 10) -> List[Restaurant]:
  """ load or generate store data, for ms1, we are generating synthetic data"""
  global _stores
  _stores = {}
  categories = ["Bakery", "Cafe", "Restaurant", "Grocery", "Deli", "Pizza", "Sushi", "Fast Food"]
  np.random.seed(42) # for reproducibility

  for i in range(num_stores):
    store_id = i+1
    name = f"Store_{store_id}"
    category = np.random.choice(categories)

    restaurant = Restaurant(store_id, name, category)

    #initialize with random but reasonable values
    restaurant.price = np.random.uniform(5.0, 25.0)
    restaurant.rating = np.random.uniform(3.0, 5.0)
    restaurant.est_inventory = np.random.randint(5, 50)
    restaurant.accuracy_score = np.random.uniform(0.7, 1.0)

    # init some historical data
    for day in range(np.random.randint(5, 20)):
      est = np.random.randint(5, 50)
      actual = int(est * np.random.uniform(0.7, 1.1)) # actual within 70-110% of estimate
      restaurant.inventory_history.append((day, est, actual))

    restaurant.calculate_accuracy()

    _stores[store_id] = restaurant

  return list[Restaurant](_stores.values())

def get_all_stores() -> List[Restaurant]:
  """return all stores"""
  return list(_stores.values())

def get_store_info(store_id: int) -> Optional[Restaurant]:
  """return store info"""
  return _stores.get(store_id)

def update_reservation(store_id: int, count: int = 1) -> bool:
  """update reservation count for a store"""
  if store_id in _stores:
    _stores[store_id].reservation_count += count
    return True
  return False

def update_exposure(store_id: int) -> bool:
  """update exposure count for a store"""
  if store_id in _stores:
    _stores[store_id].exposure_count += 1
    return True
  return False

def get_store_metrics(store_id: int) -> Optional[Dict]:
  """get store metrices as a dict"""
  if store_id not in _stores:
        return None

  store = _stores[store_id]
  return {
        'store_id': store.restaurant_id,
        'name': store.name,
        'category': store.category,
        'rating': store.rating,
        'price': store.price,
        'est_inventory': store.est_inventory,
        'actual_inventory': store.actual_inventory,
        'accuracy_score': store.accuracy_score,
        'reservation_count': store.reservation_count,
        'exposure_count': store.exposure_count,
        'completed_order_count': store.completed_order_count,
        'cancellation_count': store.cancellation_count
  }

def initialize_day(stores: List[Restaurant], actual_inventories: Optional[Dict[int, int]] = None):
  """
  initialize a new day

  if actual_inventories is provided (for testing/simulation), it will be stored and used at end_of_day_processing() when we "discover" the actual inventory
  """
  global _actual_inventories_for_day
  _actual_inventories_for_day = actual_inventories

  for store in stores:
      store.reset_daily_counters()
      # actual_inventory remains 0 until end of day when it's revealed
      # we dont know it during the day, so ranking algorithm can't use it but it could use an estimate/approximation based on history of the store


def end_of_day_processing(marketplace: Optional["Marketplace"] = None) -> Dict:
  """process end of day: calc cancellations, waste, and update accuracy"""
  global _actual_inventories_for_day
  total_cancellations = 0
  total_waste = 0
  total_revenue = 0
  total_completed = 0

  for store in _stores.values():
    # we discover the actual inventory
    # if actual_inventories were provided for testing, use those, if not generate realistic acutal inventory based on estimate and accuracy
    if _actual_inventories_for_day and store.restaurant_id in _actual_inventories_for_day:
      store.actual_inventory = _actual_inventories_for_day[store.restaurant_id]
    else:
      #generate actual inventory independently from estimate
      accuracy_factor = store.accuracy_score

      # allow actual to vary significatnly from estimate
      min_factor = 0.5 - (1.0 - accuracy_factor)*0.3 # can go as low as 20%
      max_factor = 1.5 + (1.0 - accuracy_factor) *0.5 # can go as high as 200%

      actual= int(store.est_inventory * np.random.uniform(min_factor, max_factor))
      store.actual_inventory = max(0, actual)

    # calculate cancellations
    cancellations = max(0, store.reservation_count - store.actual_inventory)
    store.cancellation_count = cancellations
    total_cancellations += cancellations

    # calculate actual sales (min of reservation and actual inventory)
    actual_sales = min(store.reservation_count, store.actual_inventory)
    store.completed_order_count = actual_sales
    total_completed += actual_sales

    # calculate waste (basically the unsold inventory)
    waste = max(0, store.actual_inventory - store.reservation_count)
    total_waste += waste

    # calculate revenue
    revenue = actual_sales * store.price
    total_revenue += revenue

    # updated inventory history with today's data
    store.inventory_history.append((
        len(store.inventory_history), #day num
        store.est_inventory,
        store.actual_inventory
    ))

    #recalculate accuracy
    store.calculate_accuracy()

    store.add_daily_summary(store.reservation_count, actual_sales)

  return{
      'total_cancellations': total_cancellations,
      'total_waste': total_waste,
      'total_revenue': total_revenue,
      'total_completed_orders': total_completed,
      'stores': {store_id: get_store_metrics(store_id) for store_id in _stores.keys()}
  }


# def get_restaurant_supply(restaurant_id):
#   #Retrieves restaurant's daily supply and price
#   # This will be random for now until we get a better generated test data
#   return {
#       'restaurant_id': restaurant_id,
#       'supply': random.randint(5, 50),
#       'price': random.uniform(5.0, 25.0)
#   }
# def get_actual_supply(restaurant_id):
#   #also will be randomly until we find better alternatives
#   return random.randint(5,50)


# def pick_emerging_restaurant(restaurants_db, min_ratings_threshold=5):
#   """Pick restaurant with low number of ratings"""
#   emerging = [r for r in restaurants_db.values() if r.number_of_ratings < min_ratings_threshold]
#   return random.choice(emerging) if emerging else None




**Customer APIs**

In [ ]:
# %%writefile customer_api.py

class Customer:
  """customer class with all required attributes"""
  def __init__(self, customer_id: int, longitude: float, latitude: float, preference_ratings: Dict[int, float], neophilia_score: float, bias_score:float,name: str = None):
    self.customer_id = customer_id
    self.name = name or f"Customer_{customer_id}"

    self.preference_ratings = preference_ratings
    self.neophilia_score = neophilia_score or 0
    self.bias_score = bias_score or 0
    self.longitude = longitude
    self.latitude = latitude
    # preferences
    self.preferences = {
        'max_price': np.random.uniform(15.0, 30.0),
        'min_rating': np.random.uniform(2.5, 4.0),
        'preferred_categories': np.random.choice(
            ["Bakery", "Cafe", "Restaurant", "Grocery", "Deli", "Pizza", "Sushi", "Fast Food"],
            size=np.random.randint(1, 4),
            replace=False
        ).tolist()
    }

    # behavioral scores
    self.satisfaction_level = np.random.uniform(0.5, 1.0) #current satisfaction

    # a bit of history tracking
    self.rating_history = [] #list of (store_id, rating)
    self.history = {
        "viewedRestaurants": [],
        "orders": []
    }

    #OPTIONAL location (might be used in our final strategy)
    self.location = None

    #decision tracking
    self.arrival_time = 0.0
    self.decision = None # 'buy' or 'leave'
    self.chosen_store_id = None
    self.displayed_stores = []
    self.learned_preference_ratings = {store_id: 0 for store_id in preference_ratings}

  def update_preferences(self, **kwargs):
    """update customer preferences"""
    self.preferences.update(kwargs)

  def add_to_history(self, store_id: int, action: str):
    """add store intereaction to history"""
    if action == 'view':
      if store_id not in self.history['viewedRestaurants']:
        self.history['viewedRestaurants'].append(store_id)
    elif action == 'order':
      self.history['orders'].append(store_id)

# global customer counter
_customer_counter = 0

def generate_customer(k: int, arrival_times: Optional[List[float]] = None) -> List[Customer]:
    """generate k customers, if arrival_times is provided, use those; other wise we generate random ones"""
    global _customer_counter

    customers = []
    np.random.seed(42 + _customer_counter) #for reproducibilty

    if arrival_times is None:
        arrival_times = sorted(np.random.uniform(0, 24, k))

    for i in range(k):
        # Generate customer with all required parameters
        _customer_counter += 1
        customer_id = _customer_counter
        longitude = np.random.uniform(-180, 180)
        latitude = np.random.uniform(-90, 90)
        preference_ratings = {}  # Will be populated if needed by generate_customer_data
        neophilia_score = np.random.uniform(0, 1)
        bias_score = np.random.uniform(-1, 1)

        customer = Customer(
            customer_id=customer_id,
            longitude=longitude,
            latitude=latitude,
            preference_ratings=preference_ratings,
            neophilia_score=neophilia_score,
            bias_score=bias_score
        )
        customer.arrival_time = arrival_times[i]
        customers.append(customer)
    return customers

def customer_arrives(customer: Customer) -> None:
  """mark customer as arrived"""
  # this function can be used for logging or tracking
  pass

def display_stores_to_customer(customer: Customer, store_list: List[Restaurant]) -> None:
  """display stores to customer and update their viewing history"""
  customer.displayed_stores = [store.restaurant_id for store in store_list]
  #update customer history
  for store in store_list:
    customer.add_to_history(store.restaurant_id, 'view')

def probability_of_purchase(store: Restaurant, customer: Customer) -> float:
  """calculate the probability that customer will purchase from this store based on prices, rating, perferecnes, and customer behavior"""
  prob = 1.0

  # price factor: lower prcies = higher probability
  if store.price > customer.preferences['max_price']:
    price_factor = 0.2 #very low probability if not fitting expected food prices for foodwaste
  else:
    price_factor = 1.0 - (store.price/customer.preferences['max_price']) * 0.5
    price_factor = max(0.1, price_factor) #minimum
  prob *= price_factor

  # rating factor: higher rating = higher probability
  if store.rating < customer.preferences['min_rating']:
    rating_factor = 0.3 #very low probability if not fitting expected
  else:
    rating_factor = store.rating/5.0
  prob *= rating_factor

  # category preferences
  if store.category in customer.preferences['preferred_categories']:
    category_factor = 1.2 # boost for preferred category
  else:
    category_factor = 0.8 #slight penalty
  prob *= category_factor

  # familiarity bias: if customer has ordered from this store before
  if store.restaurant_id in customer.history["orders"]:
    prob *= (1.0 + customer.bias_score * 0.3) # boost for familiar stores

  # neophilia: if cusotmer hasnt seen this store before
  if store.restaurant_id not in customer.history["viewedRestaurants"]:
    prob *= (1.0 + customer.neophilia_score * 0.2) # boost for new stores

  # base purchase prob (not everyone buys)
  base_prob = 0.4 # 40% base chance of buying something
  final_prob = base_prob * prob

  #clip to [0, 1]
  return max(0.0, min(1.0, final_prob))

def choose_store(stores: List[Restaurant], customer: Customer) -> Optional[Restaurant]:
  """customer chooses a store from the displayed stores, returns the chosen store or None if customer leaves"""
  if not stores:
    return None

  probabilities = []
  for store in stores:
    prob = probability_of_purchase(store, customer)
    probabilities.append(prob)

  #normalize probs
  total_prob = sum(probabilities)
  if total_prob == 0:
    return None # basically no store is attractive enough

  normalized_probs = [p/total_prob for p in probabilities]

  #decide if customer buys anything
  #overall purchase probability is the sum of individual probs
  overall_purchase_prob = min(0.85, total_prob/len(stores))
  if np.random.uniform() > overall_purchase_prob:
    #customer leaves without buying
    return None

  # customer biys from one of the stores
  chosen_idx = np.random.choice(len(stores), p=normalized_probs)
  return stores[chosen_idx]

def customer_makes_decision(customer: Customer, displayed_stores: List[Restaurant]) -> Dict:
  """customer makes a decision: buy or leave, this return a dict with action and store_id"""
  customer.displayed_stores = [store.restaurant_id for store in displayed_stores]
  chosen_store = choose_store(displayed_stores, customer)

  if chosen_store is None:
    customer.decision = 'leave'
    customer.chosen_store_id = None
    return{
        'action': customer.decision,
        'store_id': customer.chosen_store_id
    }
  else:
    customer.decision = 'buy'
    customer.chosen_store_id = chosen_store.restaurant_id
    customer.add_to_history(chosen_store.restaurant_id, 'order')
    return{
        'action': customer.decision,
        'store_id': customer.chosen_store_id
    }

def get_customer_preferences(customer: Customer) -> Dict:
  """return customer preferences as a dict"""
  return customer.preferences.copy()


### Simulation Code to generate data to test on

def generate_customer_data(num_customers: int, arrival_times: List[float], all_stores: List[Restaurant]) -> List[Customer]:
  """
  Generate customers with realistic preference clustering based on restaurant categories.
  
  This creates customer segments where each customer has higher preferences for restaurants
  in their preferred categories, and lower preferences for others. Preferences are generated
  using Gaussian distributions centered around category affinity scores.
  
  Args:
    num_customers: number of customers to generate
    arrival_times: list of arrival times for each customer
    all_stores: list of all Restaurant objects
    
  Returns:
    list of Customer objects with category-clustered preferences
  """
  customers = []
  customer_counter = 0
  np.random.seed(42)  # for reproducibility
  
  # Get unique categories and count restaurants per category
  categories = list(set(store.category for store in all_stores))
  restaurants_by_category = {cat: [s for s in all_stores if s.category == cat] for cat in categories}
  
  for i in range(num_customers):
    customer_counter += 1
    
    # Step 1: Assign customer to preferred categories (1-3 categories)
    num_preferred_categories = np.random.randint(1, min(4, len(categories) + 1))
    preferred_categories = np.random.choice(categories, size=num_preferred_categories, replace=False).tolist()
    
    # Step 2: Generate preference ratings clustered by category
    customer_preferences = {}
    
    for restaurant in all_stores:
      if restaurant.category in preferred_categories:
        # For preferred categories: use Gaussian centered at high value (mean=0.75, std=0.15)
        # This creates a cluster of high preferences around 0.6-0.9
        pref = np.random.normal(loc=0.75, scale=0.15)
      else:
        # For non-preferred categories: use Gaussian centered at low value (mean=0.35, std=0.20)
        # This creates a cluster of low preferences around 0.15-0.55
        pref = np.random.normal(loc=0.35, scale=0.20)
      
      # Clip to [0, 1] to keep preferences valid
      customer_preferences[restaurant.restaurant_id] = max(0.0, min(1.0, pref))
    
    # Step 3: Generate behavioral scores
    customer_neophilia = np.random.uniform(0, 1)
    customer_bias = np.random.uniform(-1, 1)
    
    # Step 4: Create Customer object
    customer = Customer(
        customer_id=customer_counter,
        longitude=np.random.uniform(-180, 180),  # dummy longitude
        latitude=np.random.uniform(-90, 90),     # dummy latitude
        preference_ratings=customer_preferences,
        neophilia_score=customer_neophilia,
        bias_score=customer_bias,
        name=f"Customer_{customer_counter}"
    )
    
    # Set arrival time
    customer.arrival_time = arrival_times[i]
    
    # Update preferred_categories in customer.preferences to match the clustered categories
    customer.preferences['preferred_categories'] = preferred_categories
    
    customers.append(customer)
  
  return customers

def generate_customer_probabilities(customer, restaurants:list[Restaurant], threshold=0.1):
  #This function generates the customer's purchase probability for each restaurant considering
  #the customer's preferences, preference_rating, neophilia, bias, and store price. If the app shows the customer
  #stores with purchase probability less than a threshold (0.1), the customer will not buy anything.
  scores=[]
  for restaurant in restaurants:
    # Ensure customer.preference_ratings is a dict and has the key, provide default if not
    pref_score = customer.preference_ratings.get(restaurant.restaurant_id, 0.5) # Default to 0.5 if not found
    if restaurant.price > customer.preferences['max_price']:
      price_factor=0.2
    else:
      price_factor = 1.0 - (restaurant.price/customer.preferences['max_price']) * 0.5
      price_factor = max(0.1, price_factor)
    score = pref_score* (1+customer.bias_score) * price_factor + (1- pref_score)* customer.neophilia_score * price_factor # Corrected 'pice_factor' and 'bias'
    scores.append(score)

  scores = np.array(scores)
  # Check if scores.sum is zero to avoid division by zero
  if scores.sum() == 0:
      return None
  probabilities = scores/scores.sum()
  if probabilities.max() <threshold:
    return None   #This means the customer doesn't like any of the restaurants and won't buy anything no matter what we show to him
  return probabilities

def purchase_probabilities(customers, restaurants:list[Restaurant]):
  all_probs=[]
  for customer in customers:
    probs = generate_customer_probabilities(customer, restaurants)
    all_probs.append(probs)
  return all_probs

def simulate_customer_decision(customer, displayed_stores: List[Restaurant]) -> Dict:
  customer.displayed_stores = [store.restaurant_id for store in displayed_stores]
  probs = generate_customer_probabilities(customer, displayed_stores)
  if probs is None:
    customer.decision = 'leave'
    customer.chosen_store_id = None
    return {
        'action': customer.decision,
        'store_id': customer.chosen_store_id
    }
  max_prob_idx = np.argmax(probs)
  chosen_store = displayed_stores[max_prob_idx]
  customer.decision = 'buy'
  customer.chosen_store_id = chosen_store.restaurant_id
  customer.add_to_history(chosen_store.restaurant_id, 'order')
  return{
      'action': customer.decision,
      'store_id': customer.chosen_store_id
  }


**Ranking Algorithm**
- for ms1: we will do default implemenation (show all stores) or basic implementation
-- here we ended up doing a basic ranking algorithm
- for ms2: (supposedly) we will need to agree on one strategy after trying individual strategies to see which factors to include from whose

In [ ]:
# %%writefile ranking_algorithm.py

def select_stores(customer: Customer, n: int, all_stores: List[Restaurant], t: int = 0) -> List[Restaurant]:
  """
  select n stores to display to customer

  this algorithm should optimize for:
    - revenue maximization
    - waste minimization (stores with available inventory)
    - basic fairness (exposure distribution)
    - cancellation risk reduction

  note: n is constant for all customers in a given day.

  args:
    customer: the arriving cusotmer
    n: num of stores to show
    all_stores: list of all availiable stores
    t: number of customers seen so far today (for fairness calc)

  returns:
    list of n stores to display
  """
  # if we have less than n stores, return all
  if len(all_stores) <= n:
    return all_stores

  #ms1: basic ranking algorithm
  scored_stores = []
  m = len(all_stores)
  target_exposure = t/m if m>0 and t>0 else 0

  # get normalization factors for score components
  max_price = max([s.price for s in all_stores], default=1.0)
  max_inventory = max([s.est_inventory for s in all_stores], default=1.0)

  for store in all_stores:
    # revenue componenet: price x rating (normalized)
    # higher price and rating = higher reveune potential
    revenue_score = (store.price/max_price) * (store.rating/5.0)

    # waste reduction component: available inventory
    #prefer stores with more available inventory to reduce waste
    # use safe capacity: est_inventory - reservations
    safe_capacity = max(0, store.est_inventory - store.reservation_count)
    waste_score = safe_capacity/max_inventory if max_inventory>0 else 0

    # basic fairness component: prefer underexposed stores
    # stores that have been shown less get a boost
    fairness_score = 0.0
    if t>0:
      exposure_ratio = store.exposure_count/t if t>0 else 0
      if exposure_ratio < target_exposure:
        fairness_score = (target_exposure - exposure_ratio)/(target_exposure + 1e-6)

    # cancellation risk penalty: stores with low accuracy and high load
    # penalize stores that are likely to overestimate and cause cancellations
    load = store.reservation_count/max(1, store.est_inventory)
    cancellation_risk = (1 - store.accuracy_score) * load
    cancellation_penalty = cancellation_risk

    # combined score (weighted combination)
    # current weights: revenue=0.4, waste=0.3, fairness=0.2, cancellation_penalty=-0.1
    score = (0.4 * revenue_score) + (0.3 * waste_score) + (0.2 * fairness_score) - (0.1 * cancellation_penalty)
    scored_stores.append((score, store.restaurant_id, store))

  #sort by score (decending) and return top n , x[1] is already an integer (store_id), so just use -x[1]
  scored_stores.sort(key=lambda x: (x[0], -x[1]), reverse=True)
  return [store for _, _, store in scored_stores[:n]]

# define different strategies below and label them please

#parameters to play with: 
#-cold_start: for how many customer we should recommend randomly before we have enough data to do collaborative filtering
#-learning_rate: the rate by which we update customer preferences based on their decisions

"""Base Assumption: Customers have categories preferences that influence their ratings of restaurants. Therefore, we can cluster
customers based on their category preferences and use collaborative filtering within these clusters to improve recommendation accuracy."""
class anan_strategy:
    """Anan's custom strategy for store selection"""
    def __init__(self, customer_db: List[Customer], restaurants_db: List[Restaurant]):
        self.customer_db = customer_db
        self.restaurants_db = restaurants_db
        self.arrivals_so_far = 0

    def select_stores(self, customer: Customer, n: int, cold_start: int = 10) -> List[Restaurant]:
        # Example: combine collaborative filtering with basic ranking
        if(self.arrivals_so_far < cold_start):
            basic_selected = random.sample(self.restaurants_db, n)
            return basic_selected
        
        cf_selected = self.collaborative_filtering(customer, n, self.restaurants_db, self.customer_db)
        # If less than n stores selected, fill with basic ranking
        if len(cf_selected) < n:
            remaining = n - len(cf_selected)
            basic_selected = select_stores(customer, remaining, self.restaurants_db, self.arrivals_so_far)
            # Avoid duplicates
            cf_store_ids = {store.restaurant_id for store in cf_selected}
            for store in basic_selected:
                if store.restaurant_id not in cf_store_ids:
                    cf_selected.append(store)
                    if len(cf_selected) == n:
                        break
        self.arrivals_so_far += 1
        return cf_selected

    def customer_to_dense_vector(self, customer: Customer, all_stores: List[Restaurant]) -> np.ndarray:
        """convert customer preference ratings to dense vector"""
        vector = np.zeros(len(all_stores))
        store_id_to_index = {store.restaurant_id: idx for idx, store in enumerate(all_stores)}
        for store_id, rating in customer.learned_preference_ratings.items():
            if store_id in store_id_to_index:
                index = store_id_to_index[store_id]
                vector[index] = rating
        return vector

    def cosine_similarity(self, vec1: np.ndarray, vec2: np.ndarray) -> float:
        """compute cosine similarity between two vectors"""
        if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
            return 0.0
        return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

    def collaborative_filtering(self, customer: Customer, n:int, all_stores: List[Restaurant], customer_db: List[Customer]) -> List[Restaurant]:
        """collaborative filtering based store selection with inventory and accuracy filtering"""
        customer_vector = self.customer_to_dense_vector(customer, all_stores)

        similarities = []
        for other_customer in customer_db:
            if other_customer.customer_id != customer.customer_id:
                other_vector = self.customer_to_dense_vector(other_customer, all_stores)
                sim = self.cosine_similarity(customer_vector, other_vector)
                similarities.append((sim, other_customer))

        # sort by similarity
        similarities.sort(key=lambda x: x[0], reverse=True)

        # aggregate top similar customers' preferences
        aggregate_scores = np.zeros(len(all_stores))
        for sim, similar_customer in similarities[:10]:  # consider top 10 similar customers
            similar_vector = self.customer_to_dense_vector(similar_customer, all_stores)
            aggregate_scores += sim * similar_vector

        # filter and rank stores: apply inventory and accuracy penalties
        viable_stores = []
        for idx, score in enumerate(aggregate_scores):
            store = all_stores[idx]
            
            # skip stores with no inventory
            if store.est_inventory - store.reservation_count <= 0:
                continue
            
            # apply accuracy penalty: lower accuracy = lower final score
            # accuracy_score ranges from 0-1, so multiply score by it
            adjusted_score = score * store.accuracy_score
            
            viable_stores.append((adjusted_score, idx, store))
        
        # sort by adjusted score (descending) and select top n
        viable_stores.sort(key=lambda x: x[0], reverse=True)
        selected_stores = [store for _, _, store in viable_stores[:n]]

        return selected_stores
    def update_learned_preferences(self, customer: Customer, decision: Dict, store: Restaurant, learning_rate: float = 0.1):
      """
      Update a customer's learned preference ratings based on their purchase decision
      
      args:
        customer: the customer making the decision
        decision: dict with 'action' ('buy' or 'leave') and 'store_id'
        store: the Restaurant object of the chosen store (if action is 'buy')
        learning_rate: how much to adjust preferences (0-1), default 0.1 means 10% adjustment per interaction
      
      returns:
        None (modifies customer.learned_preference_ratings in place)
      """
      
      if decision['action'] == 'buy':
        store_id = decision['store_id']
        
        # If customer bought, increase preference for that store
        # Use exponential moving average: new_pref = old_pref + learning_rate * (1 - old_pref)
        # This converges preferences toward 1.0 for purchased stores
        current_pref = customer.learned_preference_ratings[store_id]
        customer.learned_preference_ratings[store_id] = current_pref + learning_rate * (1 - current_pref)
        
        # Slightly decrease preference for stores that were displayed but not chosen
        # This helps the algorithm learn negative signals (what the customer didn't want)
        for displayed_store_id in customer.displayed_stores:
          if displayed_store_id != store_id:
            current_pref = customer.learned_preference_ratings[displayed_store_id]
            # Decay by learning_rate * (preference / 2) so we gently decrease
            customer.learned_preference_ratings[displayed_store_id] = current_pref - (learning_rate * current_pref * 0.3)
            # Keep bounded in [0, 1]
            customer.learned_preference_ratings[displayed_store_id] = max(0, min(1, customer.learned_preference_ratings[displayed_store_id]))
      
      else:  # decision['action'] == 'leave'
        # If customer left without buying, slightly decrease all displayed stores
        # since none of them were attractive enough
        for displayed_store_id in customer.displayed_stores:
          current_pref = customer.learned_preference_ratings[displayed_store_id]
          # Smaller decay for "leave" decision since we don't know which store was bad
          customer.learned_preference_ratings[displayed_store_id] = current_pref - (learning_rate * current_pref * 0.15)
          customer.learned_preference_ratings[displayed_store_id] = max(0, min(1, customer.learned_preference_ratings[displayed_store_id]))

**Simulation Module**

In [ ]:
# %%writefile simulation.py
from typing import List, Dict, Optional

class Marketplace:
  """marketplace structure containing stores, customers, and state"""

  def __init__(self, stores: List[Restaurant]):
    self.stores = stores
    self.customers: List[Customer] = []
    self.current_time = 0.0
    self.total_revenue = 0.0
    self.total_cancellations = 0
    self.total_waste = 0
    self.total_customers_seen = 0
    self.n = None # number of stores to show (constant for the day, calculated at day start)

def calculate_n(num_stores: int, expected_customers: int, total_estimated_inventory: int = None) -> int:
  """
  calculate n based on demand

  we are basing this off:
  - if demand is low (few c relative to s) show fewer stores
  - if demand is high (many c) increase n to spread customers across stores

  args:
    num_stores : total number of stores
    expected_customers: expected number of customers for the day
    total_estimated_inventory: total estimated inventory accorss all stores

  returns:
    n: number of stores to show
  """

  #base calculations: n should be proportional to demand
  # min n: at least 1 store, max n: all stores
  min_n = 1
  max_n = num_stores

  #calculate n based on customer to store ratio
  if num_stores>0:
    customers_per_store = expected_customers/num_stores
    if customers_per_store > 10: # high demand
      n = min(max_n, max(min_n, int(num_stores * 0.8))) # show 80% of stores
    elif customers_per_store > 5: # medium demand
      n = min(max_n, max(min_n, int(num_stores * 0.6))) # show 60% of stores
    else: # low demand
      n = min(max_n, max(min_n, int(num_stores * 0.4))) # show 40% of stores

    # if total inventory is provided, adjust based on inventory availability
    if total_estimated_inventory is not None:
      # more inv available ? can show more stores
      avg_inventory_per_store = total_estimated_inventory/num_stores if num_stores >0 else 0
      if avg_inventory_per_store > 30: #high inventory
        n = min(max_n, n+1)
      elif avg_inventory_per_store < 10: #low inventory
        n = max(min_n, n-1)
  else:
    n = min_n

  return max(min_n, min(max_n, n))

def initialize_marketplace(num_stores: int = 10, actual_inventories: Optional[Dict[int, int]] = None, expected_customers: int = 100) -> Marketplace:
  """ init marketplace with stores and calculate n for the day"""
  stores = load_store_data(num_stores)
  initialize_day(stores, actual_inventories)

  marketplace = Marketplace(stores)

  #calculate total estimated inv
  total_est_inventory = sum(store.est_inventory for store in stores)

  #calculate n for this day based on demand
  marketplace.n = calculate_n(num_stores, expected_customers, total_est_inventory)

  return marketplace

def simulate_customer_arrival(marketplace: Marketplace, customer: Customer) -> Dict:
  """simulate a single customer arrival and decision"""
  #mark customer as arrived
  customer_arrives(customer)
  marketplace.current_time = customer.arrival_time
  marketplace.total_customers_seen += 1

  all_stores = get_all_stores()

  #select n stores to display using ranking algorithm
  # n is constant for all customers in the day, t = number of customers seen so far (for the fairness calc)
  t = marketplace.total_customers_seen
  n = marketplace.n

  strategy
  displayed_stores = select_stores(customer, n, all_stores, t) # TODO: change this function call when testing different strategies

  #display stores to customer
  display_stores_to_customer(customer, displayed_stores)

  #update exposure for displayed
  for store in displayed_stores:
    update_exposure(store.restaurant_id)

  #customer makes decision
  decision = simulate_customer_decision(customer, displayed_stores)

  #update reservations if customer bought
  if decision['action'] == 'buy':
    update_reservation(decision['store_id'], 1)
    #get store to calculate revenue
    store = next((s for s in all_stores if s.restaurant_id == decision['store_id']), None)
    if store:
      marketplace.total_revenue += store.price

  #add customer to marketplace
  marketplace.customers.append(customer)

  return decision

def process_end_of_day(marketplace: Marketplace) -> Dict:
  """process end of day: calculate final metrics"""

  results = end_of_day_processing(marketplace)

  marketplace.total_cancellations = results['total_cancellations']
  marketplace.total_waste = results['total_waste']
  marketplace.total_revenue = results['total_revenue']
  total_customers = marketplace.total_customers_seen if marketplace else 0

  return {
    'total_cancellations': results['total_cancellations'],
    'total_waste': results['total_waste'],
    'total_revenue': results['total_revenue'],
    'total_completed_orders': results['total_completed_orders'],
    'total_customers':  results.get('total_customers', marketplace.total_customers_seen),
    'stores': results['stores']
  }

def run_simulations(num_stores: int = 10, num_customers: int = 100, n: Optional[int] = None, duration: float = 24.0, actual_inventories: Optional[Dict[int, int]] = None, verbose: bool = True) -> Dict:
  """
  run complete simulation
  args:
    num_stores: number of stores in marketplace
    num_customers: number of customers to simulate
    n: number of stores to show each customer (if None, calculated dynamically based on demand)
    duration: duration of simulation in hours (24 for a day)
    actual_inventories: optional dict mapping store_id to actual inventory
    verbose: whether to print progress

  returns:
    dict with simulation results
  """

  # initialize marketplace (calculates n is not provided)
  marketplace = initialize_marketplace(num_stores, actual_inventories, num_customers)
  if n is not None:
    marketplace.n = n
  else:
    n = marketplace.n

  if verbose:
    print(f"Initialized marketplace with {num_stores} stores")
    print(f"Simulating {num_customers} customers over {duration} hours")
    print(f"Calculated n = {n} stores to show each customer (constant for the day)\n")

  # generate customers with arrival times
  arrival_times = sorted(np.random.uniform(0, duration, num_customers))
  customers = generate_customer_data(num_customers, arrival_times, marketplace.stores)

  #process each customer
  decisions = []
  for i, customer in enumerate(customers):
    decision = simulate_customer_arrival(marketplace, customer)
    decisions.append(decision)

    if verbose and (i+1)%20 == 0:
      print(f"Processed {i+1}/{num_customers} customers...")

  # end of day processing
  if verbose:
    print("\nProcessing end of day...")
  results = process_end_of_day(marketplace)

  if verbose:
    print("\nSimulation Results:")
    print(f"Total Customers: {results['total_customers']}")
    print(f"Total Completed Orders: {results['total_completed_orders']}")
    print(f"Total Revenue: ${results['total_revenue']:.2f}")
    print(f"Total Cancellations: {results['total_cancellations']}")
    print(f"Total Waste: {results['total_waste']}")

  return {
      'marketplace': marketplace,
      'results': results,
      'decisions': decisions
  }




**Main**

This is where we will be testing.

In [ ]:
# this is where we will do all the testing
print("TEST 1: Basic Simulation")
results = run_simulations(
    num_stores=10,
    num_customers=50,
    n=None,
    duration=24.0,
    verbose=True
)

print("\nTest 1 Passed: Simulation ran successfully!")

TEST 1: Basic Simulation
Initialized marketplace with 10 stores
Simulating 50 customers over 24.0 hours
Calculated n = 4 stores to show each customer (constant for the day)

Processed 20/50 customers...
Processed 40/50 customers...

Processing end of day...

Simulation Results:
Total Customers: 50
Total Completed Orders: 50
Total Revenue: $885.98
Total Cancellations: 0
Total Waste: 189

Test 1 Passed: Simulation ran successfully!
